In [1]:
from instrumented_solvers_B import *
from test_B import get_assignment_problems, get_assignment_results, get_solvers, get_test_results

import numpy as np
import pandas as pd

In [2]:
setting_problem_stats_dict = get_assignment_results()

(min_conflicts_instrumented, max_steps=1000)(australia_map)
(min_conflicts_instrumented, max_steps=1000)(usa_csp)
(min_conflicts_instrumented, max_steps=1000)(zebra)
(min_conflicts_instrumented, max_steps=1000)(10 queens)
(min_conflicts_instrumented, max_steps=1000)(30 queens)
(min_conflicts_instrumented, max_steps=1000)(50 queens)
(min_conflicts_instrumented, max_steps=1000)(70 queens)
(min_conflicts_instrumented, max_steps=1000)(90 queens)
(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=no_inference, max_steps=1000)(australia_map)
(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=no_inference, max_steps=1000)(usa_csp)
(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=no_inference, max_steps=1000)(zebra)
(

In [18]:
#create results table
backtrack_results_df = pd.DataFrame([])
min_conflicts_results_df = pd.DataFrame([])

algo_parms = list(setting_problem_stats_dict.keys()) #primary keys
for setting in algo_parms:
    
    #figure out what solver you are using 
    solver = setting.split(",")[0][1:]
    results_per_problem = setting_problem_stats_dict[setting]
    problems = list(results_per_problem.keys()) #second level keys
    
    for problem in problems:
        results = results_per_problem[problem]
        stats = list(results.keys()) #third level keys
        
        for stat in stats:
            result_all_iter = results[stat]
                
            if stat == "assignment":
                solutions_found = np.array([not(result is None) for result in result_all_iter])
                prob_of_sucess = (np.sum(solutions_found)/10) * 100

            elif stat == "num_assignments":
                avg_num_assignments = np.mean(result_all_iter)
                sd_num_assignments = np.std(result_all_iter)
                
            elif stat == "time":
                avg_runtime = np.mean(result_all_iter)
                sd_runtime = np.std(result_all_iter)

            else: #number of repair assignments or backtracks depending on solver

                if solver == 'min_conflicts_instrumented':
                    avg_repair_assignments = np.mean(result_all_iter)
                    sd_repair_assignments = np.std(result_all_iter)
                else:
                    avg_backtracks = np.mean(result_all_iter)
                    sd_backtracks = np.std(result_all_iter)
        
        #adjust solver to give method number
        if setting == "(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=no_inference, max_steps=1000)":
            method = 1
        elif setting == "(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=forward_checking, max_steps=1000)":
            method = 2
        elif setting == "(backtracking_search_instrumented, select_unassigned_variable=first_unassigned_variable, order_domain_values=unordered_domain_values, inference=mac, max_steps=1000)":
            method = 3
        elif setting == "(backtracking_search_instrumented, select_unassigned_variable=mrv, order_domain_values=lcv, inference=no_inference, max_steps=1000)":
            method = 4
        elif setting == "(backtracking_search_instrumented, select_unassigned_variable=mrv, order_domain_values=lcv, inference=mac, max_steps=1000)":
            method = 5
        elif setting == "(min_conflicts_instrumented, max_steps=1000)":
            method = 6
        
        
        #once finished computing stats assign to correct dataframe
        if solver == 'min_conflicts_instrumented':
            row = pd.DataFrame({"solver": method,
                                "problem":problem, 
                               "avg assignments": avg_num_assignments,
                               "sd assignments": sd_num_assignments,
                               "avg repair": avg_repair_assignments,
                               "sd repair":sd_repair_assignments,
                                "avg runtime":avg_runtime,
                                "sd runtime":sd_runtime,
                               "prob sucess": prob_of_sucess}, index=range(1))
            min_conflicts_results_df = min_conflicts_results_df.append(row, ignore_index=True)
        else:
            row = pd.DataFrame({"solver": method,
                                "problem":problem, 
                   "avg assignments": avg_num_assignments,
                   "sd assignments": sd_num_assignments,
                   "avg backtracks": avg_backtracks,
                   "sd backtracks":sd_backtracks,
                    "avg runtime":avg_runtime,
                    "sd runtime":sd_runtime,
                    "prob sucess": prob_of_sucess}, range(1))
            backtrack_results_df = backtrack_results_df.append(row, ignore_index=True)
                
                          
        
    

In [19]:
#combine outputs
results_df = backtrack_results_df.append(min_conflicts_results_df, ignore_index=True)
results_df.to_excel("results.xlsx")

In [21]:
results_df.to_csv("results.csv", index=False)

5